# PAMM Pool Cross-Comparison Analysis
## Oracle Latency, Vulnerabilities & MEV Exposure

Comprehensive analysis comparing PAMM pools across:
- Oracle update latency
- Trade latency metrics
- MEV vulnerability scores
- Sandwich attack exposure
- Token pair distribution
- Validator coordination risks

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Paths
base_path = Path('../')
data_path = base_path / '01_data_cleaning/outputs/pamm_clean_final.parquet'
mev_path = base_path / '02_mev_detection/per_pamm_all_mev_with_validator.csv'

print(f"Data path exists: {data_path.exists()}")
print(f"MEV path exists: {mev_path.exists()}")

In [ ]:
# Load data
print("Loading data...")
df = pd.read_parquet(data_path)
mev_df = pd.read_csv(mev_path)

print(f"✓ Main data: {len(df):,} records, {df.shape[1]} columns")
print(f"✓ MEV data: {len(mev_df):,} records, {mev_df.shape[1]} columns")
print(f"\nMain data columns: {list(df.columns)}")
print(f"\nMEV data columns: {list(mev_df.columns)}")

## 1. Extract PAMM Pool Information

In [ ]:
# Extract pool information from trades
print("Extracting PAMM pool metrics...")

# Convert time column to datetime if needed
if 'datetime' in df.columns:
    df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# Extract trades data
trades_list = []
for idx, row in df.iterrows():
    if pd.notna(row.get('trades')) and row['trades']:
        trades = row['trades'] if isinstance(row['trades'], list) else [row['trades']]
        for trade in trades:
            if isinstance(trade, dict):
                trade_record = {
                    'slot': row['slot'],
                    'time': row['time'],
                    'datetime': row['datetime'],
                    'validator': row['validator'],
                    'signer': row['signer'],
                    'pool': trade.get('pool'),
                    'token_pair': trade.get('token_pair'),
                    'amount_in': trade.get('amount_in'),
                    'amount_out': trade.get('amount_out'),
                    'us_since_first_shred': row.get('us_since_first_shred')
                }
                trades_list.append(trade_record)
    if idx % 100000 == 0 and idx > 0:
        print(f"  Processed {idx:,} rows, extracted {len(trades_list):,} trades")

trades_df = pd.DataFrame(trades_list)
print(f"\n✓ Extracted {len(trades_df):,} trades from {len(df):,} records")
print(f"Columns: {list(trades_df.columns)}")

In [ ]:
# Extract oracle updates
print("Extracting oracle update metrics...")

oracle_list = []
for idx, row in df.iterrows():
    if pd.notna(row.get('amm_oracle')) and row['amm_oracle']:
        oracles = row['amm_oracle'] if isinstance(row['amm_oracle'], list) else [row['amm_oracle']]
        for oracle in oracles:
            if isinstance(oracle, dict):
                oracle_record = {
                    'slot': row['slot'],
                    'time': row['time'],
                    'datetime': row['datetime'],
                    'validator': row['validator'],
                    'oracle_type': oracle.get('oracle_type'),
                    'pool': oracle.get('pool'),
                    'token_pair': oracle.get('token_pair'),
                    'us_since_first_shred': row.get('us_since_first_shred')
                }
                oracle_list.append(oracle_record)
    if idx % 100000 == 0 and idx > 0:
        print(f"  Processed {idx:,} rows, extracted {len(oracle_list):,} updates")

oracle_df = pd.DataFrame(oracle_list)
print(f"\n✓ Extracted {len(oracle_df):,} oracle updates from {len(df):,} records")
print(f"Columns: {list(oracle_df.columns)}")

## 2. Calculate Oracle Latency Metrics by Pool

In [ ]:
# Calculate oracle latency per pool
print("Calculating oracle latency metrics...")

oracle_latency = oracle_df.groupby(['pool', 'token_pair']).agg({
    'us_since_first_shred': ['count', 'mean', 'median', 'std', 'min', 'max'],
    'time': lambda x: (x.max() - x.min()) if len(x) > 1 else 0
}).round(2)

oracle_latency.columns = ['update_count', 'mean_latency_us', 'median_latency_us', 
                          'std_latency_us', 'min_latency_us', 'max_latency_us', 'time_span']
oracle_latency = oracle_latency.reset_index()
oracle_latency = oracle_latency.sort_values('mean_latency_us', ascending=False)

print(f"Oracle latency metrics for {len(oracle_latency)} pool-pair combinations:")
print(oracle_latency.head(15))

In [ ]:
# Calculate trade latency per pool
print("Calculating trade latency metrics...")

trade_latency = trades_df.groupby(['pool', 'token_pair']).agg({
    'us_since_first_shred': ['count', 'mean', 'median', 'std', 'min', 'max'],
    'signer': 'nunique'
}).round(2)

trade_latency.columns = ['trade_count', 'mean_trade_latency_us', 'median_trade_latency_us', 
                         'std_trade_latency_us', 'min_trade_latency_us', 'max_trade_latency_us', 'unique_signers']
trade_latency = trade_latency.reset_index()
trade_latency = trade_latency.sort_values('mean_trade_latency_us', ascending=False)

print(f"Trade latency metrics for {len(trade_latency)} pool-pair combinations:")
print(trade_latency.head(15))

## 3. Calculate MEV Vulnerability Scores

In [ ]:
# Aggregate MEV metrics per pool
print("Calculating MEV vulnerability metrics per pool...")

mev_aggregated = mev_df.agg({
    'back_running': 'sum',
    'front_running': 'sum',
    'sandwich': 'sum',
    'fat_sandwich': 'sum',
    'sandwich_complete': 'sum',
    'cost_sol': 'sum',
    'profit_sol': 'sum',
    'net_profit_sol': 'sum',
    'confidence': 'mean'
})

print("Total MEV Summary:")
print(mev_aggregated)

# Per-pool MEV analysis
# Extract pool from amm_trade if available, or create proxy
mev_by_validator = mev_df.groupby('validator').agg({
    'back_running': 'sum',
    'front_running': 'sum',
    'sandwich': 'sum',
    'fat_sandwich': 'sum',
    'sandwich_complete': 'sum',
    'cost_sol': 'sum',
    'profit_sol': 'sum',
    'net_profit_sol': 'sum',
    'confidence': 'mean',
    'amm_trade': 'count'
}).round(4)

mev_by_validator.columns = ['back_running_count', 'front_running_count', 'sandwich_count', 
                            'fat_sandwich_count', 'sandwich_complete_count', 'total_cost_sol', 
                            'total_profit_sol', 'net_profit_sol', 'avg_confidence', 'mev_events']
mev_by_validator = mev_by_validator.sort_values('net_profit_sol', ascending=False)

print(f"\nMEV metrics by validator (top 15):")
print(mev_by_validator.head(15))

In [ ]:
# Calculate vulnerability score per token pair
print("Calculating vulnerability scores per token pair...")

# Get token pair info from trades
if len(trades_df) > 0:
    pair_metrics = trades_df.groupby('token_pair').agg({
        'trade_count': 'sum' if 'trade_count' in trades_df.columns else 'size',
        'us_since_first_shred': ['mean', 'median', 'std'],
        'signer': 'nunique',
        'validator': 'nunique',
        'pool': 'nunique'
    }).round(2)
    
    # Flatten columns
    pair_metrics.columns = ['_'.join(col).strip('_') for col in pair_metrics.columns.values]
    pair_metrics = pair_metrics.reset_index()
    
    # Rename for clarity
    if 'us_since_first_shred_mean' in pair_metrics.columns:
        pair_metrics.rename(columns={
            'us_since_first_shred_mean': 'mean_latency_us',
            'us_since_first_shred_median': 'median_latency_us',
            'us_since_first_shred_std': 'std_latency_us',
            'signer_nunique': 'unique_signers',
            'validator_nunique': 'unique_validators',
            'pool_nunique': 'unique_pools'
        }, inplace=True)
    
    # Calculate vulnerability score (higher = more vulnerable)
    pair_metrics['vulnerability_score'] = (
        (pair_metrics['mean_latency_us'] / pair_metrics['mean_latency_us'].max() * 0.3) +  # Oracle latency factor
        (pair_metrics['unique_validators'] / pair_metrics['unique_validators'].max() * 0.3) +  # Validator concentration
        (pair_metrics['unique_signers'] / pair_metrics['unique_signers'].max() * 0.2) +  # Attacker diversity
        (pair_metrics['unique_pools'] / pair_metrics['unique_pools'].max() * 0.2)  # Pool diversity
    ).round(3)
    
    pair_metrics = pair_metrics.sort_values('vulnerability_score', ascending=False)
    
    print(f"Vulnerability metrics for {len(pair_metrics)} token pairs:")
    print(pair_metrics.head(20))
else:
    print("No trade data available for token pair analysis")

## 4. Create Comparison Tables

In [ ]:
# TABLE 1: Oracle Latency Comparison
print("\n" + "="*80)
print("TABLE 1: ORACLE LATENCY METRICS BY POOL-PAIR")
print("="*80)

oracle_latency_top = oracle_latency.head(20).copy()
oracle_latency_top = oracle_latency_top[['pool', 'token_pair', 'update_count', 
                                          'mean_latency_us', 'median_latency_us', 
                                          'std_latency_us', 'max_latency_us']]
print(oracle_latency_top.to_string(index=False))
print()

In [ ]:
# TABLE 2: Trade Latency Comparison  
print("\n" + "="*80)
print("TABLE 2: TRADE LATENCY METRICS BY POOL-PAIR")
print("="*80)

trade_latency_top = trade_latency.head(20).copy()
trade_latency_top = trade_latency_top[['pool', 'token_pair', 'trade_count', 
                                        'mean_trade_latency_us', 'median_trade_latency_us', 
                                        'std_trade_latency_us', 'unique_signers']]
print(trade_latency_top.to_string(index=False))
print()

In [ ]:
# TABLE 3: Token Pair Vulnerability Scores
print("\n" + "="*80)
print("TABLE 3: TOKEN PAIR VULNERABILITY ASSESSMENT")
print("="*80)

if len(pair_metrics) > 0:
    pair_vuln_display = pair_metrics.head(20).copy()
    pair_vuln_display = pair_vuln_display[['token_pair', 'mean_latency_us', 
                                           'unique_validators', 'unique_signers', 
                                           'unique_pools', 'vulnerability_score']]
    print(pair_vuln_display.to_string(index=False))
else:
    print("No vulnerability metrics available")
print()

In [ ]:
# TABLE 4: MEV Risk by Validator
print("\n" + "="*80)
print("TABLE 4: MEV RISK ASSESSMENT BY VALIDATOR")
print("="*80)

mev_display = mev_by_validator.head(15).copy()
mev_display = mev_display.reset_index()
mev_display = mev_display[['validator', 'mev_events', 'sandwich_count', 
                           'fat_sandwich_count', 'net_profit_sol', 'avg_confidence']]
print(mev_display.to_string(index=False))
print()

## 5. Visualization: Oracle Latency Comparison

In [ ]:
# Chart 1: Top pools by oracle latency
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Sort by mean latency and take top 15
oracle_top15 = oracle_latency.nlargest(15, 'mean_latency_us').copy()
oracle_top15['pool_pair'] = oracle_top15['pool'].str[:8] + '...' + oracle_top15['token_pair']

# Plot 1: Mean latency
ax1 = axes[0]
bars1 = ax1.barh(range(len(oracle_top15)), oracle_top15['mean_latency_us'], color='steelblue')
ax1.set_yticks(range(len(oracle_top15)))
ax1.set_yticklabels(oracle_top15['pool_pair'], fontsize=9)
ax1.set_xlabel('Mean Oracle Latency (microseconds)', fontsize=11, fontweight='bold')
ax1.set_title('TOP 15 POOLS: Mean Oracle Update Latency', fontsize=12, fontweight='bold')
ax1.invert_yaxis()
for i, (idx, row) in enumerate(oracle_top15.iterrows()):
    ax1.text(row['mean_latency_us'], i, f" {row['mean_latency_us']:.0f}µs", 
            va='center', fontsize=8)

# Plot 2: Update frequency
ax2 = axes[1]
oracle_top15_freq = oracle_latency.nlargest(15, 'update_count').copy()
oracle_top15_freq['pool_pair'] = oracle_top15_freq['pool'].str[:8] + '...' + oracle_top15_freq['token_pair']
bars2 = ax2.barh(range(len(oracle_top15_freq)), oracle_top15_freq['update_count'], color='darkgreen')
ax2.set_yticks(range(len(oracle_top15_freq)))
ax2.set_yticklabels(oracle_top15_freq['pool_pair'], fontsize=9)
ax2.set_xlabel('Number of Oracle Updates', fontsize=11, fontweight='bold')
ax2.set_title('TOP 15 POOLS: Oracle Update Frequency', fontsize=12, fontweight='bold')
ax2.invert_yaxis()
for i, (idx, row) in enumerate(oracle_top15_freq.iterrows()):
    ax2.text(row['update_count'], i, f" {int(row['update_count'])}", 
            va='center', fontsize=8)

plt.tight_layout()
plt.savefig('oracle_latency_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Saved: oracle_latency_comparison.png")
plt.show()

## 6. Visualization: Trade Latency Comparison

In [ ]:
# Chart 2: Trade latency comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Mean trade latency
trade_top15 = trade_latency.nlargest(15, 'mean_trade_latency_us').copy()
trade_top15['pool_pair'] = trade_top15['pool'].str[:8] + '...' + trade_top15['token_pair']

ax1 = axes[0]
bars1 = ax1.barh(range(len(trade_top15)), trade_top15['mean_trade_latency_us'], color='coral')
ax1.set_yticks(range(len(trade_top15)))
ax1.set_yticklabels(trade_top15['pool_pair'], fontsize=9)
ax1.set_xlabel('Mean Trade Latency (microseconds)', fontsize=11, fontweight='bold')
ax1.set_title('TOP 15 POOLS: Mean Trade Execution Latency', fontsize=12, fontweight='bold')
ax1.invert_yaxis()
for i, (idx, row) in enumerate(trade_top15.iterrows()):
    ax1.text(row['mean_trade_latency_us'], i, f" {row['mean_trade_latency_us']:.0f}µs", 
            va='center', fontsize=8)

# Plot 2: Trade volume
ax2 = axes[1]
trade_top15_vol = trade_latency.nlargest(15, 'trade_count').copy()
trade_top15_vol['pool_pair'] = trade_top15_vol['pool'].str[:8] + '...' + trade_top15_vol['token_pair']
bars2 = ax2.barh(range(len(trade_top15_vol)), trade_top15_vol['trade_count'], color='darkred')
ax2.set_yticks(range(len(trade_top15_vol)))
ax2.set_yticklabels(trade_top15_vol['pool_pair'], fontsize=9)
ax2.set_xlabel('Number of Trades', fontsize=11, fontweight='bold')
ax2.set_title('TOP 15 POOLS: Trade Volume', fontsize=12, fontweight='bold')
ax2.invert_yaxis()
for i, (idx, row) in enumerate(trade_top15_vol.iterrows()):
    ax2.text(row['trade_count'], i, f" {int(row['trade_count'])}", 
            va='center', fontsize=8)

plt.tight_layout()
plt.savefig('trade_latency_comparison.png', dpi=300, bbox_inches='tight')
print("✓ Saved: trade_latency_comparison.png")
plt.show()

## 7. Visualization: Vulnerability Scores

## 8. Visualization: MEV Risk Analysis

In [ ]:
# Chart 4: MEV Risk by Validator and Attack Type
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

mev_top12 = mev_by_validator.head(12).reset_index()

# Plot 1: MEV events by type
ax1 = axes[0, 0]
x_pos = np.arange(len(mev_top12))
width = 0.2
ax1.bar(x_pos - 1.5*width, mev_top12['back_running_count'], width, label='Back-run', alpha=0.8)
ax1.bar(x_pos - 0.5*width, mev_top12['front_running_count'], width, label='Front-run', alpha=0.8)
ax1.bar(x_pos + 0.5*width, mev_top12['sandwich_count'], width, label='Sandwich', alpha=0.8)
ax1.bar(x_pos + 1.5*width, mev_top12['fat_sandwich_count'], width, label='Fat Sandwich', alpha=0.8)
ax1.set_xlabel('Validator', fontsize=11, fontweight='bold')
ax1.set_ylabel('Attack Count', fontsize=11, fontweight='bold')
ax1.set_title('MEV Attack Types by Top Validators', fontsize=12, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(mev_top12['validator'].str[:6], rotation=45, ha='right', fontsize=8)
ax1.legend(loc='upper right')
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Net Profit by Validator
ax2 = axes[0, 1]
colors_profit = ['green' if x > 0 else 'red' for x in mev_top12['net_profit_sol']]
bars2 = ax2.barh(range(len(mev_top12)), mev_top12['net_profit_sol'], color=colors_profit)
ax2.set_yticks(range(len(mev_top12)))
ax2.set_yticklabels(mev_top12['validator'], fontsize=9)
ax2.set_xlabel('Net Profit (SOL)', fontsize=11, fontweight='bold')
ax2.set_title('MEV Net Profit by Top Validators', fontsize=12, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax2.invert_yaxis()

# Plot 3: Total Cost vs Profit
ax3 = axes[1, 0]
scatter3 = ax3.scatter(mev_top12['total_cost_sol'], mev_top12['total_profit_sol'],
                       s=mev_top12['mev_events']*5, alpha=0.6,
                       c=mev_top12['net_profit_sol'], cmap='RdYlGn')
ax3.set_xlabel('Total Cost (SOL)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Total Profit (SOL)', fontsize=11, fontweight='bold')
ax3.set_title('MEV Cost vs Profit (bubble size = event count)', fontsize=12, fontweight='bold')
plt.colorbar(scatter3, ax=ax3, label='Net Profit')
ax3.grid(True, alpha=0.3)

# Plot 4: Confidence level
ax4 = axes[1, 1]
bars4 = ax4.barh(range(len(mev_top12)), mev_top12['avg_confidence'], color='steelblue')
ax4.set_yticks(range(len(mev_top12)))
ax4.set_yticklabels(mev_top12['validator'], fontsize=9)
ax4.set_xlabel('Average Detection Confidence', fontsize=11, fontweight='bold')
ax4.set_title('MEV Detection Confidence by Validator', fontsize=12, fontweight='bold')
ax4.invert_yaxis()
for i, (idx, row) in enumerate(mev_top12.iterrows()):
    ax4.text(row['avg_confidence'], i, f" {row['avg_confidence']:.3f}", 
            va='center', fontsize=8)

plt.tight_layout()
plt.savefig('mev_risk_analysis.png', dpi=300, bbox_inches='tight')
print("✓ Saved: mev_risk_analysis.png")
plt.show()

## 9. Export Summary Tables to CSV

## 10. Summary Statistics